## Import Library

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import requests
import json
import base64

## Import Dataset

In [2]:
df = pd.read_csv('clean_dataset/hotel_reservation_clean.csv', encoding='utf8')

df.head()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,1
1,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,1
2,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,0
3,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,0
4,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,0


## Split Dataset (Features and Label)

In [3]:
X = df.drop(columns=['booking_status'])
y = df['booking_status']

In [4]:
input = X.iloc[1].to_dict()
label = y.iloc[1]

print('Test row: ', input, '\n')
print('Test label: ', label)

Test row:  {'no_of_adults': 2, 'no_of_children': 0, 'no_of_weekend_nights': 2, 'no_of_week_nights': 3, 'type_of_meal_plan': 'Not Selected', 'required_car_parking_space': 0, 'room_type_reserved': 'Room_Type 1', 'lead_time': 5, 'arrival_year': 2018, 'arrival_month': 11, 'arrival_date': 6, 'market_segment_type': 'Online', 'repeated_guest': 0, 'no_of_previous_cancellations': 0, 'no_of_previous_bookings_not_canceled': 0, 'avg_price_per_room': 106.68, 'no_of_special_requests': 1} 

Test label:  1


## Make Prediction Request

In [5]:
def prepare_json(input):
    feature = {}

    for key, value in input.items():
        if isinstance(value, float):
            feature[key] = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
        elif isinstance(value, int):
            feature[key] = tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
        elif isinstance(value, str):
            feature[key] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))

    example = tf.train.Example(
        features=tf.train.Features(feature=feature)
    ).SerializeToString()

    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]

    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

json_data = prepare_json(input)

endpoint = "https://hotel-cancellation-prediction-production.up.railway.app/v1/models/hr-model:predict"
response = requests.post(endpoint, data=json_data, headers={"Content-Type": "application/json"})

prediction = response.json().get("predictions")
if prediction:
    prediction_value = prediction[0][0]
    result = "Not Cancelled" if prediction_value > 0.5 else "Cancelled"
else:
    result = "No prediction found"

print("The guest reservation status is: ", result)

The guest reservation status is:  Not Cancelled
